In [26]:
import logging
import os
import sys

import numpy as np
import pandas as pd

sys.path.insert(0, '..')

from app import db

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 50)


<ipython-input-26-f0b900f0fc61>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [27]:
db_conn = db.get_db_conn()

## Fixing missing lateral yards for receivers and qbs
...

In [13]:
def _extract_laterals(path: str) -> pd.DataFrame:
    """Download CSV from remote path."""
    return pd.read_csv(
        path,
        compression='gzip' if "csv.gz" in path else "infer",
        low_memory=False
    )


def _extract_play_by_play(db_conn, df_laterals: pd.DataFrame) -> pd.DataFrame:
    """Grab the games from the PBP that need updating"""
    game_ids = tuple(df_laterals['game_id'].unique())
    query = f"""
        SELECT * from play_by_play 
        WHERE game_id IN {game_ids}
    """
    return pd.read_sql(query, db_conn)

In [31]:
path = 'https://raw.githubusercontent.com/mrcaseb/nfl-data/master/data/lateral_rec_yards/lateral_receiving_yards_2020.csv'

df_laterals = _extract_laterals(path)
df_laterals.head()

,game_id,play_id,stat_id,lateral_rec_yards,team_abbr,player_name,gsis_player_id
0,2020_16_CIN_HOU,3932,23,7,HOU,B.Cooks,00-0031236
1,2020_10_CIN_PIT,688,23,14,PIT,C.Claypool,00-0036326
2,2020_16_CIN_HOU,3932,23,8,HOU,C.Heck,00-0036195
3,2020_09_CAR_KC,1909,23,3,CAR,C.McCaffrey,00-0033280
4,2020_06_LA_SF,1909,23,4,LA,D.Henderson,00-0035664


In [32]:
df_play_by_play = _extract_play_by_play(db_conn, df_laterals)
df_play_by_play.shape

(1114, 341)

In [55]:
df_play_by_play[
    (df_play_by_play['game_id'] == '2020_16_CIN_HOU') & 
    (df_play_by_play['play_id'] == 3932) 
]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_yards,fumble_recovery_1_player_id,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year
1112,3932,2020_16_CIN_HOU,2020122706,HOU,CIN,REG,16,HOU,home,CIN,HOU,70.0,2020-12-27,9.0,9.0,9.0,Half2,0,22.0,0,4,1.0,0,00:09,HOU 30,10,42.0,"(:09) (Shotgun) 4-D.Watson pass short left to 17-C.Hansen to HOU 38 for 8 yards. Lateral to 16-K.Coutee to HOU 29 for -9 yards. Lateral to 4-D.Watson to HOU 25 for -4 yards. Lateral to 13-B.Cooks to HOU 45 for 20 yards. Lateral to 74-M.Scharping to CIN 48 for 7 yards. Lateral to 67-C.Heck to CIN 40 for 8 yards (23-D.Phillips). FUMBLES (23-D.Phillips), ball out of bounds at CIN 40.",pass,8.0,1,0,1.0,0,0,0,short,left,11.0,-3.0,None,None,None,NaN,None,None,0,2,0.0,None,None,0.0,2.0,31,37,31.0,37.0,-6.0,31.0,37.0,-6.0,0.957789,0.001565,0.000429,0.006199,0.023571,0.000266,0.010181,0.0,0.0,0.093562,-0.093562,-9.809914,9.809914,6.69709,-6.69709,-18.785203,18.785203,0.080134,-0.173695,0.080134,-0.173695,-0.266894,0.266894,-15.259654,15.259654,0.130967,-0.130967,-13.68274,13.68274,0.016104,0.983896,0.016104,0.983896,-0.016104,0.0,1.0,0.021864,0.021864,0.169578,...,NaN,None,None,None,None,None,None,None,0.0,None,None,None,NaN,0,None,None,0.0,0.0,0.0,0.0,2020,0.687553,31.244749,63,1,First down,3932.0,13:00:00,21:06:20,NR

In [33]:
df_laterals = df_laterals[['game_id', 'play_id', 'player_name', 'lateral_rec_yards']]

df_play_by_play_2 = df_play_by_play.merge(
    df_laterals,
    how='left',
    left_on=['game_id', 'play_id', 'receiver'],
    right_on=['game_id', 'play_id', 'player_name']
)

In [34]:
df_play_by_play_2.columns

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'id', 'qb_epa', 'xyac_epa', 'xyac_mean_yardage', 'xyac_median_yardage',
       'xyac_success', 'xyac_fd', 'year', 'player_name', 'lateral_rec_yards'],
      dtype='object', length=343)

In [51]:
df_play_by_play_2[~df_play_by_play_2['lateral_rec_yards'].isna()]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,player_name,lateral_rec_yards


In [44]:
df_laterals_total = df_laterals[['game_id', 'play_id', 'lateral_rec_yards']].groupby(
    ['game_id', 'play_id']
).sum().reset_index()
df_laterals_total.rename(columns={'lateral_rec_yards': 'lateral_rec_yards_total'}, inplace=True)
df_laterals_total.head()

,game_id,play_id,lateral_rec_yards_total
0,2020_06_LA_SF,1909,4
1,2020_08_ATL_CAR,2762,3
2,2020_09_CAR_KC,1909,3
3,2020_10_CIN_PIT,688,14
4,2020_14_WAS_SF,4752,3


In [45]:
df_play_by_play_3 = df_play_by_play_2.merge(
    df_laterals_total,
    how='left',
    on=['game_id', 'play_id']
)

In [46]:
df_play_by_play_3.columns

Index(['play_id', 'game_id', 'old_game_id', 'home_team', 'away_team',
       'season_type', 'week', 'posteam', 'posteam_type', 'defteam',
       ...
       'qb_epa', 'xyac_epa', 'xyac_mean_yardage', 'xyac_median_yardage',
       'xyac_success', 'xyac_fd', 'year', 'player_name', 'lateral_rec_yards',
       'lateral_rec_yards_total'],
      dtype='object', length=344)

In [47]:
df_play_by_play_3[['game_id', 'lateral_rec_yards', 'lateral_rec_yards_total']]

,game_id,lateral_rec_yards,lateral_rec_yards_total
0,2020_10_CIN_PIT,NaN,NaN
1,2020_10_CIN_PIT,NaN,NaN
2,2020_10_CIN_PIT,NaN,NaN
3,2020_06_LA_SF,NaN,NaN
4,2020_06_LA_SF,NaN,NaN
...,...,...,...
1109,2020_16_CIN_HOU,NaN,NaN
1110,2020_16_CIN_HOU,NaN,NaN
1111,2020_16_CIN_HOU,NaN,NaN
1112,2020_16_CIN_HOU,NaN,22.0


In [50]:
df_play_by_play_3[~df_play_by_play_3['lateral_rec_yards'].isna()]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,player_name,lateral_rec_yards,lateral_rec_yards_total


In [56]:
df_play_by_play_3[~df_play_by_play_3['lateral_rec_yards_total'].isna()]

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,player_name,lateral_rec_yards,lateral_rec_yards_total
83,1909,2020_06_LA_SF,2020101811,SF,LA,REG,6,LA,away,SF,LA,72.0,2020-10-18,86.0,86.0,1886.0,Half1,0,8.0,0,2,3.0,0,01:26,LA 28,7,7.0,(1:26) (Shotgun) 16-J.Goff pass short right to 10-C.Kupp to LA 23 for -5 yards. Lateral to 27-D.Henderson pushed ob at LA 32 for 9 yards (20-J.Ward).,pass,4.0,1,0,1.0,0,0,0,short,right,-3.0,3.0,None,None,None,NaN,None,None,1,2,0.0,None,None,2.0,1.0,21,6,6.0,21.0,-15.0,6.0,21.0,-15.0,0.663945,0.100069,0.001204,0.050536,0.114864,0.000864,0.068516,0.0,0.0,0.169565,-0.348429,15.461173,-15.461173,-2.565434,2.565434,18.740209,-18.740209,-0.572879,0.224450,-0.572879,0.224450,-5.619620,5.619620,20.194432,-20.194432,-14.926485,14.926485,33.666693,-33.666693,0.160013,0.839987,0.839987,0.160013,-0.030145,0.870132,0.129868,0.196208,0.803792,-0.030535,...,None,None,None,None,None,0.0,None,None,None,NaN,0,None,None,0.0,0.0,0.0,0.0,2020,0.848588,15.141159,25,0,Punt,1909.0,20:20:00,01:42:51,Levi's Stadium,"Sunny Temp: 86° F, Humidity: 22%, Wind: NW 10 mph",10160000-0582-5952-e5bd-5825fa9c301e,2,0,PASS,0,None,None,LA 32,8,Punt,None,3.0,0:38,0.0,0.0,0.0,2.0,2.0,-5.0,KICKOFF,PUNT,01:47,01:09,LA 25,LA 32,1825.0,1933.0,16,24,Home,8,40,-2.0,51.0,1,outdoors,None,None,No

## Receiving yards adjustment

In [24]:
def _extract_receiving(db_conn) -> pd.DataFrame:
    """Getting passing stats, per player per game"""
    logging.info("Extracting receving stats by player by year from play by play...")
    query = """
        SELECT
            year,
            season_type,
            posteam AS team,
            receiver,
            week,
            defteam as opp,
            SUM(complete_pass) AS receptions,
            SUM(pass_attempt) AS targets,
            SUM(yards_gained) AS yards,
            SUM(air_yards) AS air_yards_intended,
            SUM(CASE WHEN complete_pass = 1 THEN air_yards ELSE 0 END) AS air_yards_completed,
            SUM(pass_touchdown) AS td,
            SUM(interception) as int,
            SUM(fumble) as fumbles,
            SUM(epa) AS epa
        FROM
            play_by_play_enriched
        WHERE
            (play_type = 'pass' or play_type = 'qb_spike')
            AND two_point_attempt = 0
            AND sack = 0
        GROUP BY
            year, week, defteam, posteam, receiver, season_type
        ORDER BY
            epa DESC
    """
    df = pd.read_sql(query, db_conn)
    logging.info(f"Extracted {len(df)} rows of receving stats.")
    return df

In [6]:
def _extract_laterals(db_conn) -> pd.DataFrame:
    query = """SELECT * from lateral_receiving_yards"""
    return pd.read_sql(query, db_conn)


def _extract_play_by_play(db_conn, df_laterals) -> pd.DataFrame:
    play_ids = tuple(df_laterals['play_ids'])
    game_ids = tuple(df_laterals['game_ids'])

In [7]:
df_laterals = _extract_laterals(db_conn)
df_laterals.head()

,game_id,play_id,stat_id,lateral_rec_yards,team_abbr,player_name,gsis_player_id,year
0,2020_16_CIN_HOU,3932,23,7,HOU,B.Cooks,00-0031236,2020
1,2020_10_CIN_PIT,688,23,14,PIT,C.Claypool,00-0036326,2020
2,2020_16_CIN_HOU,3932,23,8,HOU,C.Heck,00-0036195,2020
3,2020_09_CAR_KC,1909,23,3,CAR,C.McCaffrey,00-0033280,2020
4,2020_06_LA_SF,1909,23,4,LA,D.Henderson,00-0035664,2020


In [9]:
play_ids = tuple(df_laterals['play_id'])
play_ids

(3932, 688, 3932, 1909, 1909, 3932, 2762, 4752)

In [12]:
tuple(zip(df_laterals['play_id'], df_laterals['game_id']))

((3932, '2020_16_CIN_HOU'),
 (688, '2020_10_CIN_PIT'),
 (3932, '2020_16_CIN_HOU'),
 (1909, '2020_09_CAR_KC'),
 (1909, '2020_06_LA_SF'),
 (3932, '2020_16_CIN_HOU'),
 (2762, '2020_08_ATL_CAR'),
 (4752, '2020_14_WAS_SF'))

In [15]:
def _extract_play_by_play(db_conn, df_laterals) -> pd.DataFrame:
    games_and_plays = tuple(zip(df_laterals['game_id'], df_laterals['play_id']))
    query = f"""
        SELECT * FROM play_by_play_enriched 
        WHERE (game_id, play_id) in {games_and_plays}
    """
    return pd.read_sql(query, db_conn)

In [18]:
df_play_by_play = _extract_play_by_play(db_conn, df_laterals)

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,...,fumble_recovery_1_player_name,fumble_recovery_2_team,fumble_recovery_2_yards,fumble_recovery_2_player_id,fumble_recovery_2_player_name,return_team,return_yards,penalty_team,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv,season,cp,cpoe,series,series_success,series_result,order_sequence,start_time,time_of_day,stadium,weather,nfl_api_id,play_clock,play_deleted,play_type_nfl,special_teams_play,st_play_type,end_clock_time,end_yard_line,fixed_drive,fixed_drive_result,drive_real_start_time,drive_play_count,drive_time_of_possession,drive_first_downs,drive_inside20,drive_ended_with_score,drive_quarter_start,drive_quarter_end,drive_yards_penalized,drive_start_transition,drive_end_transition,drive_game_clock_start,drive_game_clock_end,drive_start_yard_line,drive_end_yard_line,drive_play_id_started,drive_play_id_ended,away_score,home_score,location,result,total,spread_line,total_line,div_game,roof,surface,temp,wind,home_coach,away_coach,stadium_id,game_stadium,success,passer,passer_jersey_number,rusher,rusher_jersey_number,receiver,receiver_jersey_number,pass,rush,first_down,aborted_play,special,play,passer_id,rusher_id,receiver_id,name,jersey_number,id,qb_epa,xyac_epa,xyac_mean_yardage,xyac_median_yardage,xyac_success,xyac_fd,year,gsis_id,position
0,1909,2020_09_CAR_KC,2020110804,KC,CAR,REG,9,CAR,away,KC,KC,32.0,2020-11-08,54.0,54.0,1854.0,Half1,0,7.0,0,2,3.0,0,00:54,KC 32,10,50.0,(:54) (Shotgun) 5-T.Bridgewater pass short right to 11-R.Anderson to KC 38 for -6 yards. Lateral to 22-C.McCaffrey to KC 29 for 9 yards (92-T.Kpassagnon).,pass,3.0,1,0,1.0,0,0,0,short,right,-2.0,2.0,None,None,None,None,None,None,3,0,0.0,None,None,0.0,3.0,13,14,14.0,13.0,1.0,14.0,13.0,1.0,0.433787,0.019809,0.000290,0.016559,0.281934,0.000530,0.247091,0.0,0.0,2.400580,-0.414874,-3.449378,3.449378,-1.352949,1.352949,-6.971780,6.971780,-1.284980,0.870106,-1.284980,0.870106,11.036465,-11.036465,-16.166177,16.166177,14.789718,-14.789718,-21.761498,21.761498,0.585543,0.414457,0.414457,0.585543,-0.035767,0.450224,0.549776,0.361024,0.638976,-0.091921,...,None,None,None,None,None,None,0.0,None,None,None,None,0,None,None,0.0,0.0,0.0,0.0,2020,0.855794,14.420605,29,0,Field goal,1909.0,13:00:00,19:24:01,Arrowhead Stadium,"cloudy Temp: 71° F, Humidity: 63%, Wind: south 19 mph",10160000-0582-9336-4180-9c8d9d1228ab,1,0,PASS,0,None,None,KC 29,7,Field goal,None,9.0,2:59,3.0,0.0,1.0,2.0,2.0,15.0,KICKOFF,FIELD_GOAL,03:45,00:46,CAR 21,KC 29,1669.0,1942.0,31,33,Home,2,64,1

In [25]:
# my_df.loc[my_df['id'].isin(ids), 'other_column'] += 1
df_receiving = _extract_receiving(db_conn)
df_receiving.head()

,year,season_type,team,receiver,week,opp,receptions,targets,yards,air_yards_intended,air_yards_completed,td,int,fumbles,epa
0,2020,REG,KC,T.Hill,12,TB,13.0,15.0,269.0,226.0,188.0,3.0,0.0,0.0,22.534387
1,2020,REG,GB,D.Adams,7,HOU,13.0,16.0,196.0,134.0,94.0,2.0,0.0,0.0,17.767565
2,2020,REG,LV,D.Waller,13,NYJ,13.0,17.0,200.0,134.0,92.0,2.0,0.0,0.0,16.420833
3,2020,REG,GB,D.Adams,9,SF,10.0,12.0,173.0,157.0,110.0,1.0,0.0,0.0,15.612783
4,2020,REG,ATL,J.Jones,6,MIN,8.0,10.0,137.0,79.0,70.0,2.0,0.0,0.0,15.549295


In [23]:
for i, row in df_laterals.iterrows():
    df_receiving[
        df_receiving['']
    ]

3932
688
3932
1909
1909
3932
2762
4752


### Ok lets try to fix receivers

- Robby Anderson - we have him for 3 extra yards
- Christian McCaffery - missing 3 yards
- Claypool - missing 14
- Cooks - missing 7
- Henderson - missing 4
- Richie James - missing 3'

Everyone but Robbie Anderson is fixed by just added the yards from the laterals table.

In [29]:
df = pd.read_sql("""SELECT * FROM play_by_play limit 10""", db_conn)
print([col for col in df.columns if '_id' in col])

['play_id', 'game_id', 'old_game_id', 'passer_player_id', 'receiver_player_id', 'rusher_player_id', 'lateral_receiver_player_id', 'lateral_rusher_player_id', 'lateral_sack_player_id', 'interception_player_id', 'lateral_interception_player_id', 'punt_returner_player_id', 'lateral_punt_returner_player_id', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'punter_player_id', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'blocked_player_id', 'tackle_for_loss_1_player_id', 'tackle_for_loss_2_player_id', 'qb_hit_1_player_id', 'qb_hit_2_player_id', 'forced_fumble_player_1_player_id', 'forced_fumble_player_2_player_id', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'assist_tackle_1_player_id', 'assist_tackle_2_player_id', 'assist_tackle_3_player_id', 'assist_tackle_4_player_id', 'pass_defense_1_player_id', 'pass_defense_2_player_id', 'fumbled_1_player_id', 'fumbled_2_player_id', 'fumble_recovery_1_player_id', 'fumble_recovery_2_player_id', 'penalty_player_id'